# the perform a mcmc sampling for Table2 1993 QS  fe_13

## for the  markov-chain monte-carlo bayesian inference model of pymc3 developed by Chris Fonnesbeck, Anand Patil, David Huard, John Salvatier

## the original analysis was done with pymc version 2

## pymc3 is what is available now it is most easily obtained from conda-forge or mini-forge

In [16]:
import os
import fnmatch
import json
import pickle
from datetime import date
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import pymc3 as pm
import theano
from theano import tensor as tt
import arviz as az
import ChiantiPy.tools.util as chutil
import ChiantiPy.model as mdl
import ChiantiPy.model.Maker as mm

In [2]:
autoreload 3

In [6]:
wd = os.getcwd()

In [7]:
today = date.today()

In [8]:
thisday =today.strftime('%Y_%B_%d')

In [9]:
thisday

'2022_April_04'

In [10]:
matplotlib qt

In [11]:
myIon = 'fe_13'

In [12]:
nameDict = chutil.convertName(myIon)

In [13]:
ls

0.test.notebooks.in.git.dir
1_fe_13_demo_make_match.html
1_fe_13_demo_make_match.ipynb
2_fe_13_demo_check_model.html
2_fe_13_demo_check_model.ipynb
3_fe_13_demo_chi2_search.ipynb
4_fe_13_demo_mcmc.ipynb
5_fe_13_demo_mcmc_trace_analyze.ipynb
fe_13_demo_make_model.html
predictPrintPrior.txt
results_2022_March_28_2000_100000.pkl
tab2_1993_qs_fe_13_1dsearch.pkl
tab2_1993_qs_fe_13_1dsearch_summary.txt
tab2_1993_qs_fe_13.json
tab2_1993_qs_fe_13_match_1dsearch.pkl
tab2_1993_qs_fe_13_match.pkl
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-21_wvl.txt
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-22_wvl.txt
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-28_wvl.txt
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-30_wvl.txt
tab2_1993_qs_fe_13_match_postPredictPrint_2000_100000_2022-03-21_wvl.txt
tab2_1993_qs_fe_13_match_postPredictPrint_2000_100000_2022-03-22_wvl.txt
tab2_1993_qs_fe_13_match_postPredictPrint_2000_100000_2022-03-28_wvl.txt
tab2_1993_

## fe_13 data is in the .json file

In [17]:
dirList = os.listdir(wd)

In [18]:
jsonList = []
for fname in dirList:
    if fnmatch.fnmatch(fname,'*.json'):
        jsonList.append(fname)

In [20]:
for idx, fname in enumerate(jsonList):
    print('idx:  %i  fname:  %s'%(idx, fname))

idx:  0  fname:  tab2_1993_qs_fe_13.json


In [21]:
jsonName = jsonList[0]

In [22]:
jsonName

'tab2_1993_qs_fe_13.json'

In [23]:
with open(jsonName, 'r') as inpt:
    specData = json.load(inpt)

In [24]:
specData.keys()

dict_keys(['filename', 'wvl0', 'intensity', 'intStd', 'dwvl', 'ions', 'ref'])

In [25]:
specData['ref']

['Brosius, Jeffrey W.; Davila, Joseph M.; Thomas, Roger J.; Monsignori-Fossi, Brunella C.',
 'Measuring Active and Quiet-Sun Coronal Plasma Properties with Extreme-Ultraviolet Spectra from SERTS ',
 '1996, Astrophysical Journal Supplement v.106, p.143',
 'ADSref:  https://ui.adsabs.harvard.edu/abs/1996ApJS..106..143B/abstract']

## the weighted chi-squared = sum ((pred-obs)/(wghtFactor x obs))**2

In [26]:
wghtFactor = 0.2

## create the box of matches object

In [27]:
box = mm.maker(specData, wghtFactor = wghtFactor, verbose = True)

 XUVTOP = /data2/a2git/chdbase/
 abundanceName = sun_photospheric_2015_scott
 minimum abundance =   1.00e+01
 # of observables / reduce # = 9        3.00


In [28]:
box.SpecData['filename']

'/data2/svn/data/serts-eunis/serts/table2-1993-QS_fe_13.txt'

In [29]:
ls

0.test.notebooks.in.git.dir
1_fe_13_demo_make_match.html
1_fe_13_demo_make_match.ipynb
2_fe_13_demo_check_model.html
2_fe_13_demo_check_model.ipynb
3_fe_13_demo_chi2_search.ipynb
4_fe_13_demo_mcmc.ipynb
5_fe_13_demo_mcmc_trace_analyze.ipynb
fe_13_demo_make_model.html
predictPrintPrior.txt
results_2022_March_28_2000_100000.pkl
tab2_1993_qs_fe_13_1dsearch.pkl
tab2_1993_qs_fe_13_1dsearch_summary.txt
tab2_1993_qs_fe_13.json
tab2_1993_qs_fe_13_match_1dsearch.pkl
tab2_1993_qs_fe_13_match.pkl
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-21_wvl.txt
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-22_wvl.txt
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-28_wvl.txt
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-30_wvl.txt
tab2_1993_qs_fe_13_match_postPredictPrint_2000_100000_2022-03-21_wvl.txt
tab2_1993_qs_fe_13_match_postPredictPrint_2000_100000_2022-03-22_wvl.txt
tab2_1993_qs_fe_13_match_postPredictPrint_2000_100000_2022-03-28_wvl.txt
tab2_1993_

## get the pickled match file name

In [30]:
pklList = []
for fname in dirList:
    if fnmatch.fnmatch(fname,'*.pkl'):
        pklList.append(fname)

In [31]:
for idx, fname in enumerate(pklList):
    print('idx:  %i  fname:  %s'%(idx, fname))

idx:  0  fname:  tab2_1993_qs_fe_13_match.pkl
idx:  1  fname:  tab2_1993_qs_fe_13_1dsearch.pkl
idx:  2  fname:  tab2_fe_13_br_summary.pkl
idx:  3  fname:  tab2_1993_qs_fe_13_match_1dsearch.pkl
idx:  4  fname:  results_2022_March_28_2000_100000.pkl


In [34]:
matchName = pklList[0]

In [35]:
matchName

'tab2_1993_qs_fe_13_match.pkl'

## open the pickled match file

In [36]:
with open(matchName,'rb') as inpt:
    match = pickle.load(inpt)

In [37]:
match.keys()

dict_keys(['match', 'Temperature', 'EDensity', 'Ndens', 'Ntemp', 'NTempDens', 'MinAbund', 'WghtFactor'])

## temperature and density are in the pickled match file

In [38]:
match['EDensity'].shape

(241,)

In [39]:
' %10.2e'%(match['EDensity'][0])

'   1.00e+06'

In [40]:
dens = match['EDensity']

In [41]:
' density range = %10.2e to %10.2e'%(dens.min(), dens.max())

' density range =   1.00e+06 to   1.00e+12'

## load the matches saved in the pickle file

In [42]:
box.loadMatch(matchName)

 EmIndices not in matchDict
Em not in matchDict
Em not in matchDict
NT not in matchDict


## demo #1 gives a prettier plot

In [43]:
plt.figure()
box.emPlot(vs='d')
plt.text(1.e+7, 3.e+26, myIon, horizontalalignment='center', fontsize=16)
plt.xlabel('Electron Density (cm$^{-3}$)', fontsize=14)
plt.ylabel('Emissin Measure (cm$^{-5}$)', fontsize=14)
plt.tight_layout()

 0  311.6
 1  312.2
 2  312.9
 3  318.1
 4  320.8
 5  321.5
 6  348.2
 7  359.7
 8  359.9


### a plot of intensity or Gofnt(T) - the usual EM loci plot but using box.emPlot

In [44]:
plt.figure()
box.emPlot(vs='d')
plt.text(1.e+7, 2.e+26, nameDict['spectroscopic'], fontsize=16)
plt.title('T = %10.2e'%(box.Temperature[0]))
plt.tight_layout()

 0  311.6
 1  312.2
 2  312.9
 3  318.1
 4  320.8
 5  321.5
 6  348.2
 7  359.7
 8  359.9


## from the plots, a density of 7.e+8 and an em of 1.e+27 is estimated

In [45]:
nDens = box.EDensity.size
print(' # of density values %5i'%(nDens))

 # of density values   241


In [46]:
densGuess = 7.e+8
Dindex = int(np.argmin(np.abs(box.EDensity - densGuess)))
' Dindex = %5i'%(Dindex)

' Dindex =   114'

### by setting the temperature by its index and the log of the EM, we can predict the spectrum

In [47]:
box.emSetIndices(Dindex)
print('density set to %10.2e '%(box.EDensity[Dindex]))

density set to   7.08e+08 


In [48]:
emGuess = 1.e+27
emLog = np.log10(emGuess)

In [49]:
box.emSetIndices(Dindex)

In [50]:
box.emSet(emLog)

## Dindex and emLog will be the starting points in the MCMC sampling

## want an em Min and Max for the MCMC sampling, guessed from the plots

In [51]:
emMinMax = [3.e+26,3.e+27]
emLogMinMax = np.log10(emMinMax)

### get predicted intensities from initial guess

In [52]:
box.predict()

## can print out the predicted intensities, as in NB #3

In [53]:
box.predictPrint(filename='priorPredictPrint.txt')

 cwd:  /home/ken/chianti/chiantipy/test_notebook
   114     7.08e+08     1.78e+06     1.00e+27
matchPkl tab2_1993_qs_fe_13_match.pkl 
wghtFactor      0.200
 -------------------------------------------------
  iwvl    IonS        wvl        Int       Pred   Int/Pred        chi
               wvl lvl1 lvl2                lower -                upper lineIdx predLine contribution
 -------------------------------------------------
     0   fe_13    311.574   6.13e+00   3.38e+00      1.814      2.244
         fe_13
           311.547    2   12        3s2 3p2 3P1.0 - 3s 3p3 3P2.0            29     0   1.000
 -------------------------------------------------
 -------------------------------------------------
     1   fe_13    312.171   1.78e+01   1.71e+01      1.042      0.202
         fe_13
           312.174    2   11        3s2 3p2 3P1.0 - 3s 3p3 3P1.0            24     0   1.000
 -------------------------------------------------
 -------------------------------------------------
     2   

## begin set up for mcmc sampling

In [54]:
nDens = match['EDensity'].size
print(' # of densities %5i'%(nDens))

 # of densities   241


## tune and samples may need to be increased after the first run and then rerun

## the no. of cores depends on your machine

In [55]:
tune = 2000
samples = 100000
cores = 4

In [56]:
padding = 10

## the predicted intensity matrix

In [57]:
pred = np.zeros((nDens, len(box.Match)), np.float64)
for iwvl, amatch in enumerate(box.Match):
    pred[0:,iwvl] += amatch['intensitySum']

In [58]:
' tune = %i  samples = %i cores = %i'%(tune, samples, cores)

' tune = 2000  samples = 100000 cores = 4'

## the observed intensities

In [59]:
nObs = len(box.Match)
intensity = np.zeros(nObs, np.float64)
for iwvl in range(nObs):
    intensity[iwvl] = box.Match[iwvl]['obsIntensity']

## create the MCMC model and perform the sampling

In [60]:
with pm.Model() as model:
    d0 = pm.DiscreteUniform('d0', lower = 0, upper = nDens - 1, dtype='int64', testval = Dindex)
    
    em = pm.Uniform('em', lower=emLogMinMax[0], upper= emLogMinMax[1], testval = emLog)  #  sigma was 0.1

    xpred = theano.shared(pred, name='p0')

    idx0 = tt.as_tensor_variable(d0)
    
    predicted = xpred[idx0]*10.**em
    
    sigma = np.sqrt(wghtFactor*intensity)
    
    Y_obs = pm.Normal('Y_obs', mu=predicted, sigma=sigma, observed=intensity)

    pm.NUTS([em],target_accept=0.87)  # was 0.8 was 0.87
    pm.Metropolis([d0], target_accept=0.87)  # was 0.8 was 0.87
    start = {'d0':Dindex}
    start['em'] = emLog
    trace = pm.sample(samples, tune=tune, cores=cores, start=start)
    pm_data = az.from_pymc3(
        trace=trace)
pm_data


/tmp/ipykernel_7280/67403462.py:20: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(samples, tune=tune, cores=cores, start=start)
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [d0]
>NUTS: [em]


Sampling 4 chains for 2_000 tune and 100_000 draw iterations (8_000 + 400_000 draws total) took 95 seconds.
The number of effective samples is smaller than 25% for some parameters.


Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> observed_data

In [61]:
pm_data.posterior.keys()

KeysView(<xarray.Dataset>
Dimensions:  (chain: 4, draw: 100000)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 ... 99994 99995 99996 99997 99998 99999
Data variables:
    d0       (chain, draw) int64 106 107 102 103 103 103 ... 104 105 105 105 108
    em       (chain, draw) float64 27.01 27.0 27.0 27.01 ... 27.04 27.04 27.02
Attributes:
    created_at:                 2022-04-04T16:05:02.015543
    arviz_version:              0.11.2
    inference_library:          pymc3
    inference_library_version:  3.11.4
    sampling_time:              95.10296487808228
    tuning_steps:               2000)

In [62]:
pm_data.posterior['em'].shape

(4, 100000)

In [63]:
pm_data.posterior['d0'].shape

(4, 100000)

In [64]:
pm_data.log_likelihood.keys()

KeysView(<xarray.Dataset>
Dimensions:      (chain: 4, draw: 100000, Y_obs_dim_0: 9)
Coordinates:
  * chain        (chain) int64 0 1 2 3
  * draw         (draw) int64 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
  * Y_obs_dim_0  (Y_obs_dim_0) int64 0 1 2 3 4 5 6 7 8
Data variables:
    Y_obs        (chain, draw, Y_obs_dim_0) float64 -5.187 -1.822 ... -1.426
Attributes:
    created_at:                 2022-04-04T16:05:12.764711
    arviz_version:              0.11.2
    inference_library:          pymc3
    inference_library_version:  3.11.4)

In [65]:
pm_data.sample_stats.keys()

KeysView(<xarray.Dataset>
Dimensions:             (chain: 4, draw: 100000)
Coordinates:
  * chain               (chain) int64 0 1 2 3
  * draw                (draw) int64 0 1 2 3 4 ... 99995 99996 99997 99998 99999
Data variables: (12/16)
    perf_counter_start  (chain, draw) float64 6.555e+03 6.555e+03 ... 6.649e+03
    diverging           (chain, draw) bool False False False ... False False
    accept              (chain, draw) float64 3.494 0.7899 0.5242 ... 1.0 0.7629
    max_energy_error    (chain, draw) float64 0.152 0.1056 ... 1.248 -0.69
    accepted            (chain, draw) bool True True True ... False True True
    lp                  (chain, draw) float64 -33.33 -33.83 ... -35.3 -34.09
    ...                  ...
    energy_error        (chain, draw) float64 -0.01877 0.1056 ... 0.0 -0.6891
    process_time_diff   (chain, draw) float64 0.0003924 0.000198 ... 0.0003511
    perf_counter_diff   (chain, draw) float64 0.0003925 0.000198 ... 0.0003511
    tree_depth          (cha

In [66]:
pm_data.observed_data.keys()

KeysView(<xarray.Dataset>
Dimensions:      (Y_obs_dim_0: 9)
Coordinates:
  * Y_obs_dim_0  (Y_obs_dim_0) int64 0 1 2 3 4 5 6 7 8
Data variables:
    Y_obs        (Y_obs_dim_0) float64 6.13 17.8 7.34 6.09 ... 55.0 28.4 11.8
Attributes:
    created_at:                 2022-04-04T16:05:12.765863
    arviz_version:              0.11.2
    inference_library:          pymc3
    inference_library_version:  3.11.4)

## save the trace for later analysis

## here, the trace will be used for analysis and not the pm_data

In [67]:
pth = pm.save_trace(trace)

In [68]:
pth

'.pymc_2.trace'

## can come back later an load the trace but it will need the model available 

## can create a model that skips the sampling

trace = pm.load_trace(pth, model)

## arviz can plot the traces and histograms, among other things

In [69]:
az.plot_trace(pm_data, var_names=["d0"])

array([[<AxesSubplot:title={'center':'d0'}>,
        <AxesSubplot:title={'center':'d0'}>]], dtype=object)

In [70]:
az.plot_trace(pm_data, var_names=["em"])

array([[<AxesSubplot:title={'center':'em'}>,
        <AxesSubplot:title={'center':'em'}>]], dtype=object)

In [71]:
trace.stat_names

{'accept',
 'accepted',
 'depth',
 'diverging',
 'energy',
 'energy_error',
 'max_energy_error',
 'mean_tree_accept',
 'model_logp',
 'perf_counter_diff',
 'perf_counter_start',
 'process_time_diff',
 'scaling',
 'step_size',
 'step_size_bar',
 'tree_size',
 'tune'}

In [72]:
trace.varnames

['d0', 'em_interval__', 'em']

In [73]:
trace['d0'].shape

(400000,)

## get the mean values and std of the traces

In [74]:
d0Mean = trace['d0'].mean()
d0Std = trace['d0'].std()
' d0Mean = %i d0Std = %10.5f'%(d0Mean, d0Std)

' d0Mean = 105 d0Std =    2.24893'

## Dindex was our initial guess

In [75]:
Dindex

114

In [76]:
plt.figure()
xyhist = plt.hist(trace['d0'])
plt.xlabel('Temperature Index', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.title('d0    mean = %8.3f  std = %8.5f \n tune = %i  samples = %i wF:  %6.3f'%(d0Mean, d0Std, tune, samples, wghtFactor), fontsize=14)
plt.tight_layout()

In [77]:
em0Mean = trace['em'].mean()
em0Std = trace['em'].std()
' em0Mean = %10.3f em0Std = %10.5f'%(em0Mean, em0Std)

' em0Mean =     27.012 em0Std =    0.01564'

## emLog was our initial guess

In [78]:
emLog

27.0

In [79]:
plt.figure()
xyhist = plt.hist(trace['em'])
plt.xlabel('Log$_{10}$ Emission Measure', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.title('EM    mean = %8.3f  std = %8.5f \n tune = %i  samples = %i wF: %6.3f'%(em0Mean, em0Std, tune, samples, wghtFactor), fontsize=14)
plt.tight_layout()

## save the results to a file

In [80]:
results = {}
results['dMean'] = d0Mean
results['dStd'] = d0Std
results['emMean'] = em0Mean
results['emStd'] = em0Std
results['tune'] = tune
results['samples'] = samples
results['date'] = thisday
results['wghtFactor'] = wghtFactor
results['pth'] = pth
results['emMinMax'] = emMinMax
results['emLogMinMax'] = emLogMinMax

In [81]:
resultsName = 'results_%s_%i_%i.pkl'%(thisday,tune,samples)

In [82]:
resultsName

'results_2022_April_04_2000_100000.pkl'

In [83]:
with open(resultsName,'wb') as outpt:
    pickle.dump(results,outpt)

## predict the intensities with the parameters derived from the MCMC sampling

In [84]:
newDindex = int(np.round(d0Mean))

In [85]:
newDindex

105

In [86]:
' newDindex %i  priorDindex  %i'%(newDindex, Dindex)

' newDindex 105  priorDindex  114'

In [87]:
newEmLog = em0Mean

In [88]:
' new Emlog:  %10.3f   prior EMlog:  %10.3f'%(newEmLog, emLog)

' new Emlog:      27.012   prior EMlog:      27.000'

In [89]:
box.emSetIndices([newDindex])
print('density  set to %10.2e '%(dens[newDindex]))

density  set to   4.22e+08 


In [90]:
box.EmIndices

array([105])

In [91]:
box.emSet([newEmLog])

In [92]:
box.predict()

In [93]:
sort = 'wvl'

In [94]:
matchName

'tab2_1993_qs_fe_13_match.pkl'

In [95]:
os.path.splitext(matchName)

('tab2_1993_qs_fe_13_match', '.pkl')

In [96]:
printName = os.path.splitext(matchName)[0] + '_postPredictPrint_%i_%i_%s_%s.txt'%(tune, samples, today, sort)

In [97]:
printName

'tab2_1993_qs_fe_13_match_postPredictPrint_2000_100000_2022-04-04_wvl.txt'

In [98]:
box.predictPrint(filename=printName, sort=sort)

 cwd:  /home/ken/chianti/chiantipy/test_notebook
   105     4.22e+08     1.78e+06     1.03e+27
matchPkl tab2_1993_qs_fe_13_match.pkl 
wghtFactor      0.200
 -------------------------------------------------
  iwvl    IonS        wvl        Int       Pred   Int/Pred        chi
               wvl lvl1 lvl2                lower -                upper lineIdx predLine contribution
 -------------------------------------------------
     0   fe_13    311.574   6.13e+00   2.88e+00      2.131      2.654
         fe_13
           311.547    2   12        3s2 3p2 3P1.0 - 3s 3p3 3P2.0            29     0   1.000
 -------------------------------------------------
 -------------------------------------------------
     1   fe_13    312.171   1.78e+01   1.94e+01      0.916      0.460
         fe_13
           312.174    2   11        3s2 3p2 3P1.0 - 3s 3p3 3P1.0            24     0   1.000
 -------------------------------------------------
 -------------------------------------------------
     2   

In [99]:
diffName = os.path.splitext(matchName)[0] + '_postDiffPrint_%i_%i_%s_%s.txt'%(tune, samples, today, sort)

In [100]:
diffName

'tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-04-04_wvl.txt'

In [101]:
box.diffPrint(filename=diffName, sort=sort)

   105     4.22e+08     1.78e+06     1.03e+27
 -------------------------------------------------
 cwd:  /home/ken/chianti/chiantipy/test_notebook
 today is 2022_April_04
 WghtFactor =      0.200
 index      density  temperature           Em
   105     4.22e+08     1.78e+06     1.03e+27   27.012
 -------------------------------------------------
 chi = abs(int - pred)/(wght*int))  strDiff = (int - pred)/pred
                        A                                                    abs        abs
  iwvl    ionS        wvl  intensity  predicted   int/pred        chi     relDev    dif/int
     0   fe_13    311.574   6.13e+00   2.88e+00      2.131      2.654      1.131      0.531
     1   fe_13    312.171   1.78e+01   1.94e+01      0.916      0.460     -0.084     -0.092
     2   fe_13    312.907   7.34e+00   6.21e+00      1.183      0.772      0.183      0.154
     3   fe_13    318.129   6.09e+00   9.21e+00      0.661      2.565     -0.339     -0.513
     4   fe_13    320.802   2.45e+01 

## plot the differences

In [102]:
wvl = box.Diff['wvl']
diff = box.Diff['diffOverInt']

In [103]:
diffMean = diff.mean()
diffStd = diff.std()

In [104]:
mytitle = 'diff Mean %10.3f  diff Std  %10.3f'%(diffMean, diffStd)

In [105]:
mytitle

'diff Mean      0.008  diff Std       0.268'

In [106]:
plt.figure()
plt.plot(wvl, diff,'o')

In [107]:
plt.axhline(diffMean, color='k', lw=2, label='Mean')
plt.axhline(diffMean + diffStd, color='r', lw=2, linestyle='--', label='1 std')
plt.axhline(diffMean - diffStd, color='r', lw=2, linestyle='--')  #, label='1 std')
plt.axhline(diffMean + 2.*diffStd, color='b', lw=2, linestyle='dotted', label='2 std')
plt.axhline(diffMean - 2.*diffStd, color='b', lw=2, linestyle='dotted')  #, label='2 std')
plt.axhline(diffMean + 3.*diffStd, color='g', lw=2, linestyle='dotted', label='3 std')
plt.axhline(diffMean - 3.*diffStd, color='g', lw=2, linestyle='dotted')  #, label='3 std')

In [108]:
plt.xlabel('Wavelength ($\AA$)', fontsize=14)
plt.ylabel(r'(Obs - Pred)/(w $\times$ Obs)', fontsize=14)

Text(30.972222222222214, 0.5, '(Obs - Pred)/(w $\\times$ Obs)')

In [109]:
plt.title(mytitle, fontsize=14)

Text(0.5, 1.0, 'diff Mean      0.008  diff Std       0.268')

In [110]:
plt.legend(loc='upper right', bbox_to_anchor=(0.99, 1.0), fontsize=12)

In [111]:
plt.tight_layout()

## this can also be done with diffPlot

In [116]:
box.diffPlot(title=True)

## the plot object are saved in the DiffPlot attribute

In [117]:
box.DiffPlot.keys()

dict_keys(['fig', 'ax'])

In [118]:
fig = box.DiffPlot['fig']
ax =  box.DiffPlot['ax']

## so, can do things like the following

In [119]:
ax.set_xlim([310., 330.])

(310.0, 330.0)

In [120]:
fig.tight_layout()